# Clasificador de priorización

## Objetivo:

El clasificador tiene como objetivo predicir si un documento es de prioridad alta o no.

### 1.- Creamos un ranking de algoritmos 

#### Definimos las rutas

In [ ]:
# Ruta de la data entrenamiento
path_data = "training_data/TRAIN_INTER.tsv" 
# Ruta donde guardaremos los resultados de la evaluacion de nuestros modelos
path_result = "result/result_final.csv" 
# Ruta donde guarderemos el clasificador con mejor evaluación
path_result_cls_transport = "results_models/cls_prioridad.joblib"

#### Creamos el ranking 

In [ ]:
# Pasamos como datos de entradas las rutas definidas a anteriormente y el nro de datos (filas) a considerar.
# revisar las rutas para ver los resultados

max_classification_random(path_data, path_result, path_result_cls_transport, 1000)
    print("--- %s seconds ---" % (time.time() - start_time))
    print("Finish")

### 2.- Evaluamos los 4 mejores algoritmos con sus respectivos parametros

#### Creamos un metodo llamado "Test_model" que se encargar de entrenar nuestros modelos e imprimir la matriz de confusion.

In [ ]:
def test_model(model, X_train, y_train, X_test, y_test, ngram_range, use_idf):
    count_vectorizer = CountVectorizer(
        analyzer='word',
        tokenizer=tokenize,
        lowercase=True,
        stop_words=spanish_stopwords,
        decode_error='ignore',
        ngram_range=ngram_range
    )

    text_cls = Pipeline([
        ('vect', count_vectorizer),  # strings to token integer counts
        ('tfidf', TfidfTransformer(use_idf=use_idf)),  # integer counts to weighted TF-IDF scores
        ('cls', model)  # train on TF-IDF vectors w/ Naive Bayes classifier
    ])

    text_cls.fit(X_train, y_train)
    predicted = text_cls.predict(X_test)
    result = metrics.classification_report(y_test, predicted)
    print(result)
    print(metrics.confusion_matrix(y_test, predicted))
    return text_cls

#### Definimos nuestros modelos que deseamos entrenar, visualizamos resultados  y guardamos los modelos.

In [ ]:
subject_corpus_train = create_subject_corpus(path_corpus_train)

    msg_train = subject_corpus_train['subject']
    label_train = subject_corpus_train['priority']
    subject_corpus_test = create_subject_corpus(path_corpus_test)
    msg_test = subject_corpus_test['subject']
    label_test = subject_corpus_test['priority']
    
    # MODELO 01
    print("RandonForest")
    model = RandomForestClassifier(criterion='gini', max_features='log2', n_estimators=150)
    model_predict = test_model(model, msg_train, label_train, msg_test, label_test, (1, 1), False)
    joblib.dump(model_predict, "results_models_test/RandonForest.joblib")
    
    # MODELO 02
    print("LinerSVC")
    model = svm.LinearSVC(C=2, loss='squared_hinge')
    model_predict = test_model(model, msg_train, label_train, msg_test, label_test, (1, 2), True)
    joblib.dump(model_predict, "results_models_test/LinerSVC.joblib")
    
    # MODELO 03
    print("MultinomialNB")
    model = MultinomialNB(alpha=0.01)
    model_predict = test_model(model, msg_train, label_train, msg_test, label_test, (1, 2), False)
    joblib.dump(model_predict, "results_models_test/MultinomialNB.joblib")
    
    # MODELO 04
    print("SVC")
    model = svm.SVC(C=100, kernel='rbf', probability=True)
    model_predict = test_model(model, msg_train, label_train, msg_test, label_test, (1, 2), False)
    joblib.dump(model_predict, "results_models_test/SVC.joblib")